### In this exercise, first, you will familiarize yourself with python libraries statsmodels and SciPy that let you do statistical analysis: calculate descriptive statistics, sample the. datapoints, study relationship between variables, and perform hypothesis testing. Then, second, you will work independently on a set of exercises that will let you practice your skills.

### Here we show a couple of examples of how to do common analyses, but there are many more useful statistical distributions and statistical tests. In the linked documentation, you will find complete information, and a lot more details compared to what is outlined in this notebook:
- [statsmodels](https://www.statsmodels.org/stable/examples/index.html), [more complete documentation](https://www.statsmodels.org/stable/stats.html)
- [SciPy](https://docs.scipy.org/doc/scipy/reference/stats.html)

# Tutorial: How to use the libraries

- You will be working with the full US 2015 census dataset (acs2015_county_data.csv, available at https://www.kaggle.com/muonneutrino/us-census-demographic-data#acs2015_county_data.csv). Here we load it. Take some time to familiarize yourself with the contents. One row represents one county.

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math

#load the statistical libraries
from statsmodels.stats import diagnostic
from scipy import stats

In [2]:
data_folder = './data/'
df = pd.read_csv(data_folder + 'acs2015_county_data.csv')

In [3]:
df.columns

Index(['CensusId', 'State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'Citizen', 'Income',
       'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment'],
      dtype='object')

In [4]:
df.sample(10)

,CensusId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
226,6081,California,San Mateo,748731,368416,380315,25.3,40.8,2.5,0.1,...,2.5,2.6,5.0,26.8,387006,79.8,11.7,8.3,0.2,6.7
2276,42065,Pennsylvania,Jefferson,44756,22240,22516,0.8,97.5,0.5,0.2,...,4.1,0.9,2.8,22.9,19756,82.0,10.5,7.2,0.3,6.4
2597,48149,Texas,Fayette,24849,12267,12582,19.6,72.6,7.2,0.0,...,1.7,1.2,4.5,23.3,11669,77.4,13.4,9.0,0.2,1.8
1582,29201,Missouri,Scott,39061,19062,19999,2.1,84.1,11.3,0.3,...,0.7,0.7,4.1,18.6,17280,81.1,12.0,6.7,0.2,6.8
1209,24035,Maryland,Queen Anne's,48600,24024,24576,3.4,86.8,7.4,0.0,...,1.6,0.9,6.4,34.7,25305,72.6,19.8,7.6,0.1,5.3
2121,39157,Ohio,Tuscarawas,92697,45715,46982,2.3,95.2,0.8,0.3,...,2.0,1.6,2.5,21.3,42879,84.5,10.0,5.3,0.2,7.0
1623,30051,Montana,Liberty,2369,1160,1209,0.8,96.2,0.0,1.1,...,21.3,0.9,15.7,11.0,950,56.7,18.2,15.3,9.8,4.5
378,12117,Florida,Seminole,437346,211546,225800,18.8,64.0,10.8,0.1,...,0.8,1.8,6.1,26.9,210450,82.9,11.6,5.4,0.1,9.4
808,19039,Iowa,Clarke,9280,4686,4594,11.4,87.9,0.1,0.0,...,3.6,0.3,4.2,23.6,4504,82.3,11.1,6.2,0.5,5.0
1472,28143,Mississippi,Tunica,10477,4916,5561,1.3,21.1,75.5,0.0,...,2.4,0.0,1.0,21.7,4191,82.6,15.6,1.7,0.1,12.4


- We will focus first on income per capita across counties.

In [5]:
ax = df['IncomePerCap'].hist(bins = 50)

### 1. Calculating descriptive statisctics

- Let's calculate basic descriptive statistics of the income per capita

In [6]:
df['IncomePerCap'].describe()

count     3220.000000
mean     23981.771739
std       6204.339960
min       5878.000000
25%      20238.500000
50%      23460.000000
75%      27053.250000
max      65600.000000
Name: IncomePerCap, dtype: float64

- What distribution does the data come from? Here is how we can test the goodness of the fit.

In [7]:
# does the data come from a normal distrbution?
diagnostic.kstest_normal(df['IncomePerCap'].values, dist = 'norm')

(0.0637621121184162, 0.0009999999999998899)

- p_value < 0.05 -> we can reject the null hypothesis that the data comes from a normal distribution!

In [8]:
#how about exponential?
diagnostic.kstest_normal(df['IncomePerCap'].values, dist = 'exp')

(0.433243770582646, 0.0009999999999998899)

- p < 0.05 -> not exponential either!

### 2. How to sample the data

- Often we need to find random samples from the dataset. We can do so conveniently with pandas wrappers.

In [9]:
#make 10 samples with replacement
sample1_counties = df.sample(n = 10, replace = True)

#make 10 samples without replacement
sample1_counties = df.sample(n = 10, replace = False)

#sometimes we want to sample in an ublanaced way, so that we upsample datapoints of certain characteristic,
#and downsample the others. this can be achieved with weights parameter
#here we sample by upsampling counties with large population
sample2_counties = df.sample(n = 10, replace = False, weights = df['TotalPop'])

In [10]:
#on avergage, the samples in the sample produce with reweighting now have higher population, as we wanted!
print(sample1_counties['TotalPop'].mean())
print(sample2_counties['TotalPop'].mean())

50016.5
550091.8


### 3. Examining relationship between two variables

- Is income per capita of a county correlated with the unemployment?

In [11]:
stats.pearsonr(df['IncomePerCap'],df['Employed'])

(0.2646136320394489, 9.942215354237806e-53)

- There is a small (0.26), but significant (p < 0.05) positive correlation

In [12]:
stats.spearmanr(df['IncomePerCap'],df['Employed'])

SpearmanrResult(correlation=0.30770631560595474, pvalue=1.415296431173735e-71)

- Spearman rank coorrelation is also significant, and slightly higher

### 4. Hypothesis testing

- Is income per capita higher in New York counties compared to California counties?

In [13]:
df.loc[df['State'] == 'New York']['IncomePerCap'].mean()

28189.75806451613

In [14]:
df.loc[df['State'] == 'California']['IncomePerCap'].mean()

27902.603448275862

- We see that there is a ~300$ gap. Quite a lot!
- But is it significantly higher? Let's use a t-test. This is a two-sided test for the null hypothesis that the two independent samples have identical average (expected) values. 

In [15]:
stats.ttest_ind(df.loc[df['State'] == 'New York']['IncomePerCap'], df.loc[df['State'] == 'California']['IncomePerCap'])

Ttest_indResult(statistic=0.19788117232375713, pvalue=0.8434785239169611)

- p is not less than 0.05 -> we cannot reject the null hypothesis that the income is the same -> there is no significant difference

### 5. How to measure uncertainty
- Now we see the importance of mesuring uncertainty and indicating it on data visualizations.
- Visual inspection of 95% confidence intervals lets us see that the difference is not significant (the errorbars are overlapping)

(25000.0, 32000.0)

In [16]:
ax = sns.barplot(x="State", y="IncomePerCap", data=df.loc[df['State'].isin(['New York','California'])])
plt.ylim([25000,32000])

# Exercise 1: Boosting the economy by incentivizing self-employment

Assume the biggest priority of the local government in 2018 is to increase per-capita income. To do so, the officials plan to adopt a strategy for incentivizing self-employment through a series of campaigns, educational programs, and dedicated funds.

Since it is unethical and impossible in this setting to run a controlled experiment involving citizens (e.g., fire employees and force them to self-employ), the officials have asked you, the data scientist, to establish the effect of self-employment on the economy, relying on observational data.

**A)** Using suitable methods, determine and quantify the dependency between the percentage of self-employed citizens and per capita income across all 3,212 US counties. Do citizens in counties that have a higher percentage of self-employed people earn more per capita?

In [17]:
stats.pearsonr(df['SelfEmployed'],df['IncomePerCap'])

(0.08727386609551788, 7.052861196521423e-07)

- There is a small (0.08), but significant (p < 0.05) positive correlation

In [18]:
stats.spearmanr(df['SelfEmployed'],df['IncomePerCap'])

SpearmanrResult(correlation=0.056413673053189874, pvalue=0.0013623847564714516)

- Spearman rank coorrelation is also significant, and slightly smaller

In [19]:
per_capita_self_empl = df[['State','IncomePerCap', 'SelfEmployed']]
sns.lmplot(x='SelfEmployed',y='IncomePerCap', data=per_capita_self_empl)
plt.xlabel("Percentage of Self Employed people [%]")
plt.ylabel("Income per Capita [$]")
plt.show()

**there is no clear dependency, very small positive correlation.**

**B)** The pilot program will involve all counties within a limited set of three US states. Set A includes Wisconsin, Tennessee, and  Minnesota. Quantify the dependency of per-capita income on self-employment rates across all the counties in set A.

In [20]:
setA = ['Wisconsin', 'Tennessee', 'Minnesota']
setAdf = df.loc[df['State'].isin(setA)]

In [21]:
stats.pearsonr(setAdf['SelfEmployed'],setAdf['IncomePerCap'])

(-0.202293507365215, 0.0011881639322139396)

- There is a very weak negative dependency, almost no dependency.

In [22]:
per_capita_self_empl_setA = setAdf[['State','IncomePerCap', 'SelfEmployed']]
sns.lmplot(x='SelfEmployed',y='IncomePerCap', data=per_capita_self_empl_setA)
plt.xlabel("Percentage of Self Employed people [%]")
plt.ylabel("Income per Capita [$]")
plt.show()

**C)** In which state within set A is the observed effect of self-employment on per-capita income the strongest?

In [23]:
sns.lmplot(x='SelfEmployed',y='IncomePerCap', data=setAdf, hue = 'State')
plt.xlabel("Percentage of Self Employed people [%]")
plt.ylabel("Income per Capita [$]")
plt.ylim([10000,50000])
plt.xlim([0,22])
plt.show()

In [24]:
Wisconsin_per_capita_self_empl = setAdf.query("State == 'Wisconsin'") 
Tennessee_per_capita_self_empl = setAdf.query("State == 'Tennessee'") 
Minnesota_per_capita_self_empl = setAdf.query("State == 'Minnesota'")

print("Wisconsin",stats.pearsonr(Wisconsin_per_capita_self_empl['SelfEmployed'],Wisconsin_per_capita_self_empl['IncomePerCap']))
print("Tennessee",stats.pearsonr(Tennessee_per_capita_self_empl['SelfEmployed'],Tennessee_per_capita_self_empl['IncomePerCap']))
print("Minnesota",stats.pearsonr(Minnesota_per_capita_self_empl['SelfEmployed'],Minnesota_per_capita_self_empl['IncomePerCap']))

Wisconsin (-0.32905300016378525, 0.004768134887745222)
Tennessee (-0.23836048684913147, 0.02001163195552807)
Minnesota (-0.2538551921654062, 0.01766519930091192)


**In Wisconsin, the nagative dependency is the strongest.**

**D)** Set B includes New Jersey, Kansas, and Rhode Island. Repeat the analysis from steps B and C above, but now for set B. In which of the two sets A and B (if any) would you recommend incentivizing self-employment? Explain your reasoning. Hint: It is useful to add a notion of confidence to your results and explore the data visually.

In [25]:
setB = ['New Jersey', 'Kansas', 'Rhode Island']
setBdf = df.query("State == 'New Jersey' | \
                    State == 'Kansas' | \
                    State == 'Rhode Island' ") 

In [26]:
per_capita_self_empl_setB = setBdf[['State','IncomePerCap', 'SelfEmployed']]

sns.lmplot(x='SelfEmployed',y='IncomePerCap', data=per_capita_self_empl_setB)
plt.xlabel("Percentage of Self Employed people [%]")
plt.ylabel("Income per Capita [$]")
plt.ylim([10000,60000])
plt.xlim([0,25])
plt.show()

In [27]:
stats.pearsonr(setBdf['SelfEmployed'],setBdf['IncomePerCap'])

(-0.2322214134608569, 0.007607493042857402)

In [94]:
stats.spearmanr(setBdf['SelfEmployed'],setBdf['IncomePerCap'])

SpearmanrResult(correlation=-0.16010624610827853, pvalue=0.06773890774775308)

In [28]:
sns.lmplot(x='SelfEmployed',y='IncomePerCap', data=per_capita_self_empl_setB, hue = 'State')
plt.xlabel("Percentage of Self Employed people [%]")
plt.ylabel("Income per Capita [$]")
plt.ylim([10000,60000])
plt.xlim([0,25])
plt.show()

In [29]:
Kansas_per_capita_self_empl = per_capita_self_empl_setB.query("State == 'Kansas'") 
New_Jersey_per_capita_self_empl = per_capita_self_empl_setB.query("State == 'New Jersey'") 
Rhode_island_per_capita_self_empl = per_capita_self_empl_setB.query("State == 'Rhode Island'")

print("Kansas",stats.pearsonr(Kansas_per_capita_self_empl['SelfEmployed'],Kansas_per_capita_self_empl['IncomePerCap']))
print("New Jersey",stats.pearsonr(New_Jersey_per_capita_self_empl['SelfEmployed'],New_Jersey_per_capita_self_empl['IncomePerCap']))
print("Rhode Island",stats.pearsonr(Rhode_island_per_capita_self_empl['SelfEmployed'],Rhode_island_per_capita_self_empl['IncomePerCap']))

Kansas (0.2581029971443692, 0.00785335184683246)
New Jersey (0.48044784713362815, 0.027488944757806236)
Rhode Island (0.9021788382149805, 0.03618310201575917)


There is a weak negative dependency overall, moderate to strong positive when disaggregated.
This is a Simpson’s Paradox: the effect is reversed when the data is aggregated.
We would recommend in set B, not in set A (there is significant positive correlation within
all states separately in set B).

# Exercise 2. Data visualization and read the stats comprehension and interpretation questions.

- As part of the two homeworks and the final exam, you will be at times asked to interpret your result based on the understanding of underlying concepts. The questions below can help you practice and get a sense of how well you understand the concepts covered in the class so far. Can you answer these questions confidently?

### Question 1. The yearly salaries (in thousands of CHF) of 7000 people are summarized in the following box plot. If the 20th percentile of the salaries is 110,000, how many people earn between 110,000 and 114,000?

- a) 2100
- b) 700
- c) 350
- d) 70

<img src="boxplot.png" style="width: 400px;" align="left"/>

the 25th percentile is 114,000, so we have 5% of the data between 110,000 and 114,000.
so 5% x 7000 = 350 persons earn between 110,000 and 114,000.

### Question 2. What visualizations are suitable for getting insights about the distribution of a single continuous variable?

- a) Barplot and histogram
- b) Boxplot and histogram
- c) Scatterplot and boxplot
- d) Barplot, boxplot, and histogram

b) Boxplot and histogram

### Question 3. In a new groundbreaking study, 28 out of 100 patients reported improvements after taking a new medicine. It is known that, when given a placebo, 20% of the patients report feeling better. What is the p-value assuming the null hypothesis that the probability of successful treatment in this groundbreaking study is the same as the probability of reporting feeling better under placebo, according to a one-sided binomial test? Hint: you may use the statsmodels.stats.proportion.binom_test function.

- a) 0.00034
- b) 0.0034
- c) 0.034
- d) 0.34

In [48]:
from statsmodels.stats.proportion import binom_test
binom_test(28, 100, prop=0.2, alternative='larger')

0.034151629639075066

c) 0.034

### Question 4. The 95% confidence interval of the average height of 1000 people using bootstrap resampling with 10000 bootstrap samples is calculated by:

- a) Sampling 1000 height values with replacement and computing the mean. This is repeated 10000 times to create a sorted list of the 10000 means. The CI is defined by the 500th and the 9500th value in sorted order.
- b) Sampling 10000 height values without replacement and computing the mean. This is repeated 1000 times to create a sorted list of the 1000 means. The CI is defined by the 25th and the 975th value in sorted order.
- c) Sampling 1000 height values with replacement and computing the mean. This is repeated 10000 times to create a sorted list of the 10000 means. The CI is defined by the 250th and the 9750th value in sorted order.
- d) Sampling 1000 height values without replacement and computing the mean. This is repeated 10000 times to create a sorted list of the 10000 means. The CI is defined by the 500th and the 9500th value in sorted order.

### Implement your bootstrapping function that you will use in the rest of the course. It should take an array and the number of iterations as inputs, and output 95% confidence intervals of the mean.

The correct answer: c) Sampling 1000 height values with replacement and computing the mean.
This is repeated 10000 times to create a sorted list of the 10000 means. 
The CI is defined by the 250th and the 9750th value in sorted order.

In [ ]:
# load dataset
data = read_csv('pima-indians-diabetes.data.csv', header=None)
values = data.values
# configure bootstrap
n_iterations = 1000
n_size = int(len(data) * 0.50)
# run bootstrap
stats = list()
for i in range(n_iterations):
	# prepare train and test sets
	train = resample(values, n_samples=n_size)
	test = numpy.array([x for x in values if x.tolist() not in train.tolist()])
	# fit model
	model = DecisionTreeClassifier()
	model.fit(train[:,:-1], train[:,-1])
	# evaluate model
	predictions = model.predict(test[:,:-1])
	score = accuracy_score(test[:,-1], predictions)
	print(score)
	stats.append(score)

In [83]:
def bootstrap_CI(data, bootsraps_n, estimator=np.mean):
    """creates bootstrap samples, computes the estimates and returns the 95% confidence interval.
       params:
           - data: the data which to sample
           - bootsraps_n: bootsrap samples size
           - estimator: the func to use (mean, std, ...)
           
       return:
           - the estimates (np.array)
           - 95% confidence intervall as tuple (p_025, p_975)
    """
    
    # Create an empty array to store the estimates
    estimates = np.empty(bootsraps_n)
    
    # Create bootstrap estimates
    for i in range(bootsraps_n):
        sample_i = np.random.choice(data,size=len(data))
        estimates[i] = estimator(sample_i)
    
    return ( estimates, (np.percentile(estimates,[2.5])[0],np.percentile(estimates,[97.5])[0] ) )

In [84]:
test_df = df[['State','IncomePerCap']]
estimates, CI = bootstrap_CI(test_df['IncomePerCap'], bootsraps_n=1000)

In [85]:
CI

(23764.482717391304, 24184.43950310559)

In [86]:
test_df['IncomePerCap'].mean()

23981.771739130436

In [93]:
# Plot probability density function
sns.displot(estimates, kde=True)
plt.axvline(x=CI[0], ymin=0, ymax=1,label='2.5th percentile',c='y')
plt.axvline(x=CI[1], ymin=0, ymax=1,label='97.5th percentile',c='r')
plt.xlabel("IncomePerCap")
plt.ylabel("PDF")
plt.title("Probability Density Function")
plt.legend()
plt.show()

### Question 5. A study conducted at EPFL revealed a negative Spearman’s correlation between GPA and entry-level job salary. Which of the following statements is true?

- a) There could be a positive Spearman’s correlation between GPA and entry-level job salary in every single department of EPFL.
- b) Spearman correlation between GPA and entry-level job salary in every single department has to be positive.
- c) Across all levels, as opposed to entry-level only, higher GPA scores are associated with higher job salaries.
- d) We can infer that finishing EPFL with a higher GPA causes you to have a lower entry-level job salary.

a) There could be a positive Spearman’s correlation between GPA and entry-level job salary in every single department of EPFL.

### Question 6. The average number of requests per page during a day on a large website is 200, while the median is 15. 

- a) You can assume that the distribution is heavy-tailed
- b) You are dealing with a left-skewed distribution
- c) Half of the pages get more than 200 requests
- d) None of the above

The correct answer: d) None of the above. The data is not necessarily heavy-tailed nor skewed, a signle outlier
could shift the average.